# Use Case 2: Data from multiple sources in an animal experiment
<span style = "font-size:22px">This notebook illustrates the usage of the `vitabel` class to visualize, annotate and process time-series data from the medical field. Please find the detailed, searchable documentation here: 
[![Documentation Status](https://readthedocs.org/projects/vitabel/badge/?version=latest)](https://vitabel.readthedocs.io/en/latest/?badge=latest)<br>
In this case we analyze data collected in an animal laboratory experiment of cardiopulmonary resuscitation. This notebook in particular demonstrates the capabilities of vitabel to **align** time-series data recorded by **multiple devices** with unsynchronized clocks.</span>

In [ ]:
from vitabel import Vitals, IntervalLabel

from pathlib import Path
import pandas as pd

## 1) Load Data

<span style = "font-size:18px">We begin by specifying the file paths for the multiple data sources. In this use case, three different types of 
files are read: data from the mechanical CPR device is stored in an XML export from a proprietary format. Invasive blood pressure was recorder from a patient monitor via [VitalRecorder](https://doi.org/10.1038/s41598-018-20062-4). Airflow, airway pressure were recorded each by a seperates singleblaord computer and stored in CSV files. </span>

In [ ]:
lucas_file = Path("data/Lucas_file_Lucas.xml") # exported via CodeStat
vital_recorder_file = "./data/vital_file.vit" # recorded via VitalRecorder (https://vitaldb.net/vital-recorder/) 

flow_file = Path("data/flow.csv.bz2")
airway_pressure_file = "data/p1.csv.bz2"
capnography_file = "data/capno.csv.bz2"

<span style = "font-size:18px">A new instance of the pyvital class is initialized and all data is loaded from the files.<br>
As ventilatory parameters were recorded in unix time `time_start` and `time_unit` must be defined accordingly.
</span>

In [ ]:
case = Vitals()
case.add_defibrillator_recording(lucas_file)
case.add_vital_db_recording(vital_recorder_file, 
                            metadata = {"source": "GE Healthcare monitor"}) # providing metadata to the channels

case.add_data_from_csv(
    flow_file,
    time_start = pd.Timestamp(1970, 1, 1, 0, 0, 0),
    time_unit = "ms",
    metadata = {"source": "volucapno"},
    index_col = "timestamp",
)
case.add_data_from_csv(
    capnography_file,
    time_start = pd.Timestamp(1970, 1, 1, 0, 0, 0),
    time_unit = "ms",
    metadata = {"source": "volucapno"},
    index_col = "Timestamp",
)
case.add_data_from_csv(
    airway_pressure_file,
    time_start = pd.Timestamp(1970, 1, 1, 0, 0, 0),
    time_unit = "ms",
    metadata = {"source": "volucapno"},
    index_col = 0,
    names = ["airway_pressure", "temperature_1"],
)

## 2) Process Data

<span style = "font-size:18px">Several channels are called and **renamed** to our discretion.</span>

In [ ]:
capno_channel = case.get_channel("CO2 Concentration")
capno_channel.rename("capnography")

ibp_channel = case.get_channel("GE alt/IBP1")
ibp_channel.rename("invasive blood pressure")

case.get_channel("GE alt/PLETH").rename("ppg") # just another way to acieve the same

<span style = "font-size:18px">We get an overview over all channels by calling the `get_channel_infos` routine</span>

In [ ]:
case.get_channel_infos()

<span style = "font-size:18px">The remaining channels are called. </span>

In [ ]:
cc_channel = case.get_channel("cc")
flow_channel = case.get_channel("airflow")
pressure_channel = case.get_channel("airway_pressure")

<span style = "font-size:18px">We generated a  new `IntervalLabel` to later annotate noisy segements of the invasive blood pressure recordings, for example due to sampling of blood gas samples. </span>

In [ ]:
aline_noise = IntervalLabel("IBP noise")
aline_noise.attach_to(ibp_channel)

<span style = "font-size:18px">`print_data_summary` gives us information about channels and labels and their **attachment**. <br>Labels attached to a channel are shifted together with it.</span>

In [ ]:
case.print_data_summary()

## 3) Interactively Plot Data


<span style = "font-size:18px">The plotstyle of the channels is adapted by the `set_channel_plotstyle` function. All keyword-arguments of [axes.plot](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.plot.html) in matplotlib can be stored in the plotstyle dictionary. </span>

In [ ]:
case.set_channel_plotstyle(capno_channel, color = "goldenrod", lw = 1, alpha = 0.8)
case.set_channel_plotstyle("airflow", color = "blue", lw = 1, alpha = 0.8) # another way to call the chanel by its name
case.get_channel("airway_pressure").plotstyle = {"color" : "green", "lw" : 1, "alpha" : 0.8} # yet another less preferred way 
case.set_channel_plotstyle(ibp_channel, color = "red", lw = 1, alpha = 0.8)
case.set_channel_plotstyle(cc_channel, color = "purple", marker = "o", alpha = 0.8, linestyle = "", label = "chest compressions") # be aware, that we define here the label displayed in the plot legend
case.set_label_plotstyle(aline_noise, color = "dimgray", lw = 3, alpha = 0.8)

<span style = "font-size:18px">An interactive plot is initialized. The menu an explanation to **align** the channels is given in the tab `Align Timelines`.<br>
We can first align the chest compressions to the IBP and than select both and align them with the first artefact of a chest compression in the airway pressure. <br>
Afterward, we can **label** noisy segments in the blood-pressure signal the the menu offered in the `Annotate`-Tab. Be aware that we have added an `IntervaLabel` which has to be defined by a start and an end, thus clicking twice.</span>

In [ ]:
plot = case.plot_interactive(
    channels = [
        [flow_channel, "cc"], # again calling the channel by its name (s.above)
        [capno_channel],
        [pressure_channel],
        [ibp_channel],
    ],
    labels = [[], [], [], ["IBP noise"]],
    channel_overviews = [[ibp_channel]],
    time_unit = "s",
    subplots_kwargs = {"figsize": (16.5, 8)},
)

plot

## 4) Store Data


In [ ]:
case.save_data("case_2.json")

In [ ]:
case.info()